In [3]:
import os
import psycopg2
import pandas as pd
from PIL import Image
import torch
from transformers import CLIPProcessor, CLIPModel
from pymilvus import MilvusClient

# ================== Config ==================
DB_PARAMS = {
    "dbname": "video_frame",   # database đã tạo
    "user": "postgres",
    "password": "123",         # đổi theo mật khẩu của bạn
    "host": "localhost",
    "port": "5432"
}

CSV_DIR   = r"D:\Big_project_2025\Video_Similarity_Search\data\csv"
FRAME_DIR = r"D:\Big_project_2025\Video_Similarity_Search\data\key_frame"
VIDEO_DIR = r"D:\Big_project_2025\Video_Similarity_Search\data\video"
MODEL_DIR = r"D:\Big_project_2025\huggingface_cache"  # nơi lưu model CLIP

# ================== Kết nối DB + Milvus ==================s
conn = psycopg2.connect(**DB_PARAMS)
cur = conn.cursor()
client = MilvusClient(uri="http://localhost:19530")


d:\Big_project_2025\venv\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.27.2 is exactly one major version older than the runtime version 6.31.1 at schema.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
d:\Big_project_2025\venv\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.27.2 is exactly one major version older than the runtime version 6.31.1 at common.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
d:\Big_project_2025\venv\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.27.2 is exactly one major version older than the runtime version 6.31.1 at milvus.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
d:\Big_project_2025\venv\Lib\site-packages\google

# Cell 2. Tạo bảng PostgreSQL (chạy 1 lần)
chạy bên pg admin rồi

# Cell 3 – Tạo collection Milvus (chạy 1 lần)

In [9]:
# Kết nối Milvus
milvus_client = MilvusClient(uri="http://localhost:19530")

# Tên collection
collection_name = "text_image_video_collection"

# Xóa collection cũ (nếu có) -> khởi tạo lại từ đầu
if milvus_client.has_collection(collection_name):
    milvus_client.drop_collection(collection_name)
    print(f"⚠️ Collection '{collection_name}' đã bị xóa")

# Tạo collection mới
milvus_client.create_collection(
    collection_name=collection_name,
    dimension=512,              # Vector size của CLIP ViT-B/32
    auto_id=True,               # Tự động tạo ID
    enable_dynamic_field=True   # Cho phép thêm field động (vd: frame_path)
)

# Kiểm tra danh sách collection hiện có
collections = milvus_client.list_collections()
print("✅ Danh sách collection:", collections)


✅ Danh sách collection: ['video_search', 'image_collection', 'text_image_video_collection', 'Movies', 'my_rag_collection']


# Cell 4 – Load CLIP model

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32", cache_dir=MODEL_DIR).to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32", cache_dir=MODEL_DIR)

def encode_image(image_path):
    try:
        image = Image.open(image_path)
        inputs = processor(images=image, return_tensors="pt").to(device)
        with torch.no_grad():
            image_features = model.get_image_features(**inputs)
        return image_features[0].cpu().numpy()
    except Exception as e:
        print(f"Lỗi khi xử lý {image_path}: {e}")
        return None


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


# Cell 5 – Hàm insert tách riêng

In [16]:
# Insert Milvus
def insert_to_milvus(frame_path, emb):
    res = client.insert(
        collection_name="text_image_video_collection",
        data=[{"vector": emb, "frame_path": frame_path}]
    )
    # Trả về ID đầu tiên
    return res["ids"][0]

# Insert PostgreSQL
def insert_to_postgres(video_id, frame_path, pts_time, frame_idx, fps, milvus_id):
    cur.execute("""
        INSERT INTO frame_mappings (video_id, frame_path, pts_time, frame_idx, fps, milvus_id)
        VALUES (%s,%s,%s,%s,%s,%s)
        ON CONFLICT (frame_path) DO NOTHING
    """, (video_id, frame_path, pts_time, frame_idx, fps, milvus_id))


# Cell 6 – Index toàn bộ video (dùng khi mới chạy lần đầu hoặc re-index)

In [18]:
# ================== Cell 6: Index dữ liệu ==================
# Hàm insert vào Milvus
def insert_to_milvus(frame_path, emb):
    try:
        res = client.insert(
            collection_name="text_image_video_collection",
            data=[{"vector": emb, "frame_path": frame_path}]
        )
        # Milvus client v2 trả về dict, primary key nằm trong "ids"
        if "ids" in res and len(res["ids"]) > 0:
            return res["ids"][0]
        return None
    except Exception as e:
        print(f"❌ Lỗi insert Milvus cho {frame_path}: {e}")
        return None

# Hàm insert vào Postgres
def insert_to_postgres(video_id, frame_path, pts_time, frame_idx, fps, milvus_id):
    try:
        cur.execute("""
            INSERT INTO frame_mappings (video_id, frame_path, pts_time, frame_idx, fps, milvus_id)
            VALUES (%s,%s,%s,%s,%s,%s)
            ON CONFLICT (frame_path) DO NOTHING
        """, (video_id, frame_path, pts_time, frame_idx, fps, milvus_id))
    except Exception as e:
        print(f"❌ Lỗi insert Postgres cho {frame_path}: {e}")

# Hàm index toàn bộ video
def index_videos():
    for key_frame_dir in os.listdir(FRAME_DIR):
        frame_dir_path = os.path.join(FRAME_DIR, key_frame_dir)
        csv_path = os.path.join(CSV_DIR, f"{key_frame_dir}.csv")
        video_path = os.path.join(VIDEO_DIR, f"{key_frame_dir}.mp4")

        # Kiểm tra file cần thiết
        if not (os.path.isdir(frame_dir_path) and os.path.exists(video_path) and os.path.exists(csv_path)):
            print(f"⚠️ Thiếu file cho {key_frame_dir}, bỏ qua.")
            continue

        # Đọc mapping CSV
        mapping_df = pd.read_csv(csv_path)

        # Insert video nếu chưa có
        cur.execute("""
            INSERT INTO videos (video_path, title, description)
            VALUES (%s, %s, %s)
            ON CONFLICT (video_path) DO NOTHING
            RETURNING id
        """, (video_path.replace("\\","/"), os.path.basename(video_path), "Video demo"))

        row = cur.fetchone()
        if row:  # nếu insert mới thành công
            video_id = row[0]
        else:    # nếu đã có -> lấy id cũ
            cur.execute("SELECT id FROM videos WHERE video_path=%s", (video_path.replace("\\","/"),))
            row = cur.fetchone()
            if row:
                video_id = row[0]
            else:
                print(f"❌ Không tìm thấy video {video_path}, bỏ qua.")
                continue

        # Duyệt từng frame trong thư mục
        for frame_file in os.listdir(frame_dir_path):
            if not frame_file.endswith((".jpg", ".jpeg", ".png")):
                continue
            frame_path = os.path.join(frame_dir_path, frame_file).replace("\\","/")

            # Lấy index frame từ tên file
            try:
                frame_idx = int(os.path.splitext(frame_file)[0])
            except:
                continue

            # Lấy row mapping
            row = mapping_df[mapping_df["n"] == frame_idx]
            if row.empty:
                continue

            pts_time = float(row["pts_time"].values[0])
            fps = int(row["fps"].values[0])

            # Encode ảnh
            emb = encode_image(frame_path)
            if emb is None:
                continue

            # Insert Milvus
            milvus_id = insert_to_milvus(frame_path, emb)

            # Insert Postgres
            insert_to_postgres(video_id, frame_path, pts_time, frame_idx, fps, milvus_id)

        # Commit sau mỗi video
        conn.commit()
        print(f"✅ Indexed {key_frame_dir}")

# ================== Chạy index ==================
index_videos()


✅ Indexed L21_V001
✅ Indexed L21_V002
✅ Indexed L21_V003


# Cell 7 – Index incremental (chỉ data mới)

In [ ]:
def index_new_data():
    for key_frame_dir in os.listdir(FRAME_DIR):
        frame_dir_path = os.path.join(FRAME_DIR, key_frame_dir)
        csv_path = os.path.join(CSV_DIR, f"{key_frame_dir}.csv")
        video_path = os.path.join(VIDEO_DIR, f"{key_frame_dir}.mp4")

        if not (os.path.isdir(frame_dir_path) and os.path.exists(video_path) and os.path.exists(csv_path)):
            continue

        mapping_df = pd.read_csv(csv_path)

        # Lấy video_id
        cur.execute("SELECT id FROM videos WHERE video_path=%s", (video_path,))
        video_id = cur.fetchone()
        if not video_id:
            continue
        video_id = video_id[0]

        # Check frame mới
        for frame_file in os.listdir(frame_dir_path):
            if not frame_file.endswith((".jpg",".jpeg",".png")):
                continue
            frame_path = os.path.join(frame_dir_path, frame_file).replace("\\","/")

            cur.execute("SELECT 1 FROM frame_mappings WHERE frame_path=%s", (frame_path,))
            if cur.fetchone():  # đã có thì bỏ qua
                continue

            try:
                frame_idx = int(os.path.splitext(frame_file)[0])
            except:
                continue

            row = mapping_df[mapping_df["n"] == frame_idx]
            if row.empty:
                continue

            pts_time = float(row["pts_time"].values[0])
            fps = int(row["fps"].values[0])

            # Encode
            emb = encode_image(frame_path)
            if emb is None:
                continue

            # Insert Milvus + Postgres
            milvus_id = insert_to_milvus(frame_path, emb)
            insert_to_postgres(video_id, frame_path, pts_time, frame_idx, fps, milvus_id)

        conn.commit()
        print(f"✅ Indexed new frames for {key_frame_dir}")


# Với cấu trúc này bạn có thể:

- Chạy Cell 2 + 3 chỉ 1 lần duy nhất để khởi tạo DB và Collection.

- Sau đó dùng Cell 6 để index tất cả data cũ.

- Về sau chỉ cần chạy Cell 7 để nạp thêm data mới vào, không ảnh hưởng dữ liệu cũ.

# cell 8

Đảm bảo chạy Cell 4 trước Cell 8

In [5]:
# ================== Cell 8: Search API (Text Search) ==================
import torch.nn.functional as F

# Encode text thành embedding
def encode_text(text):
    try:
        inputs = processor(
            text=[text],
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=77
        ).to(device)
        with torch.no_grad():
            text_features = model.get_text_features(**inputs)
        return F.normalize(text_features, p=2, dim=1)[0].cpu().numpy()
    except Exception as e:
        print(f"❌ Lỗi encode text: {e}")
        return None

# Chuyển giây -> mm:ss
def format_time(seconds):
    m = int(seconds // 60)
    s = int(seconds % 60)
    return f"{m}:{s:02d}"

# Gom nhiều timestamp gần nhau thành khoảng
def group_timestamps(timestamps, gap_threshold=10.0):
    if not timestamps:
        return []
    timestamps = sorted(timestamps)
    ranges = []
    start = timestamps[0]
    end = timestamps[0]

    for t in timestamps[1:]:
        if t - end <= gap_threshold:
            end = t
        else:
            ranges.append((start, end))
            start = t
            end = t
    ranges.append((start, end))
    return ranges

# Search video theo text
def search_videos_by_text(text_query, top_k=10, gap_threshold=10.0):
    text_emb = encode_text(text_query)
    if text_emb is None:
        return []

    # Search Milvus
    results = client.search(
        collection_name="text_image_video_collection",
        data=[text_emb],
        limit=top_k,
        output_fields=["frame_path"]
    )

    # Gom kết quả theo video
    output = []
    for hit in results[0]:
        frame_path = hit["entity"]["frame_path"]

        # Lấy info từ Postgres
        cur.execute("""
            SELECT v.video_path, v.title, fm.pts_time
            FROM frame_mappings fm
            JOIN videos v ON fm.video_id = v.id
            WHERE fm.frame_path=%s
        """, (frame_path,))
        row = cur.fetchone()
        if row:
            video_path, title, pts_time = row
            output.append((video_path, title, pts_time))

    # Gom timestamp theo video
    grouped_results = {}
    for video_path, title, pts_time in output:
        if video_path not in grouped_results:
            grouped_results[video_path] = {"title": title, "timestamps": []}
        grouped_results[video_path]["timestamps"].append(pts_time)

    # Chuyển thành khoảng thời gian
    final_results = []
    for video_path, data in grouped_results.items():
        time_ranges = group_timestamps(data["timestamps"], gap_threshold=gap_threshold)
        final_results.append({
            "video_path": video_path,
            "title": data["title"],
            "time_ranges": [(format_time(s), format_time(e)) for s, e in time_ranges]
        })

    return final_results



## dịch từ việt sang anh để queey nè

In [1]:
from deep_translator import GoogleTranslator

# Văn bản cần dịch
text_vi = "bác sĩ mổ trong phòng mỗ với áo màu xanh"

# Dịch từ Tiếng Việt sang Tiếng Anh
result = GoogleTranslator(source='vi', target='en').translate(text_vi)

print("Tiếng Việt:", text_vi)
print("English:", result)


Tiếng Việt: bác sĩ mổ trong phòng mỗ với áo màu xanh
English: The doctor operated in the room with a blue shirt


In [6]:
from deep_translator import GoogleTranslator

# Văn bản cần dịch
text_vi = "nhiều con cá đang bơi"

# Dịch từ Tiếng Việt sang Tiếng Anh
result = GoogleTranslator(source='vi', target='en').translate(text_vi)

# ================== Demo ==================
query = result  # thay bằng text cần tìm
results = search_videos_by_text(query, top_k=20, gap_threshold=15.0)

print(f"📌 Kết quả tìm kiếm cho: \"{query}\"")
for res in results:
    print(f"\n🎬 Video: {res['title']}")
    print(f"📂 Path: {res['video_path']}")
    for start, end in res["time_ranges"]:
        print(f"👉 Xuất hiện từ {start} đến {end}")

📌 Kết quả tìm kiếm cho: "Many fish are swimming"

🎬 Video: L21_V001.mp4
📂 Path: D:/Big_project_2025/Video_Similarity_Search/data/video/L21_V001.mp4
👉 Xuất hiện từ 6:02 đến 6:02
👉 Xuất hiện từ 17:11 đến 17:49
👉 Xuất hiện từ 18:07 đến 18:11
👉 Xuất hiện từ 19:22 đến 19:31

🎬 Video: L21_V003.mp4
📂 Path: D:/Big_project_2025/Video_Similarity_Search/data/video/L21_V003.mp4
👉 Xuất hiện từ 3:51 đến 4:07
👉 Xuất hiện từ 5:57 đến 6:06
👉 Xuất hiện từ 9:37 đến 9:49


## 📝 Phân tích lại pipeline

1. **Video → Frame → Embedding**

   * Video được chia thành **các keyframe**.
   * Mỗi frame có metadata được lưu vào CSV/DB:

     * **n**: số thứ tự keyframe.
     * **pts\_time**: thời điểm xuất hiện frame trong video (tính bằng giây).
     * **fps**: frame per second (ví dụ: 30 fps).
     * **frame\_idx**: chỉ số frame trong toàn video (fps × thời gian).

   👉 Ví dụ:

   ```
   n=2, pts_time=3.0, fps=30.0, frame_idx=90
   ```

   nghĩa là frame số 90 (tại giây thứ 3.0) được chọn làm keyframe thứ 2.

2. **Index & Lưu trữ**

   * Embedding của từng frame được tính bằng CLIP.
   * Vector này được lưu vào Milvus cùng metadata: `{frame_path, video_id, pts_time, frame_idx}`.

3. **Tìm kiếm (Text → Embedding → Milvus)**

   * Người dùng nhập text (VD: `"fish swimming"`).
   * Text được encode thành vector bằng CLIP.
   * Milvus tìm trong vector database → trả về top-K frame gần nhất.
   * PostgreSQL/CSV được query theo **frame\_idx hoặc pts\_time** để biết frame này nằm ở giây nào, thuộc video nào.
   * Kết quả cuối cùng: **video + khoảng thời gian chứa frame**.



## 🔎 Ý nghĩa của bảng `n, pts_time, fps, frame_idx`

* Bảng này giúp **mapping giữa embedding và video gốc**.
* Khi Milvus trả về `frame_idx = 1131`, ta có thể dùng bảng để biết:

  * Frame này thuộc **giây 37.7** (pts\_time).
  * Trong video gốc có thể lấy đoạn từ **\[37.7s → 40s]** để hiển thị.
* Đây chính là **bước quan trọng để không chỉ trả về hình ảnh tĩnh, mà còn hiển thị đoạn video đúng ngữ cảnh**.
